In [141]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from ta.trend import SMAIndicator
import yfinance as yf
import pandas as pd
import json

import numpy as np

class finance_manager:
    def __init__(self) -> None:
        self.data = {}

    def get_data(self,company_id,meashure,_start_period,_end_period):
        self.data[company_id]=yf.download(company_id,interval=meashure,start=_start_period,end=_end_period)
        self.data[company_id] = self.data[company_id].fillna(method='bfill')
        self.data[company_id] = self.data[company_id].fillna(method='ffill')

    def print_data(self,company_id,*param):
        temp_df = self.data[company_id].copy()
        fig = go.Figure()
        for item in param:
            if("position" in item):
                fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item],name=item))
            elif("ret_acum" in item):
                fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item],name=item))
            else:
                fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item],name=item))
                
        fig.show()

class khalman_signal_filter:
    def __init__(self,new_info=0.5,old_info=0.5):
        self.new_info = new_info
        self.old_info = old_info

    def filter(self,Data:pd.DataFrame):
        temp_vector = []
        self.y_n=Data[0]
        self.y_n1=Data[0]
        for x_0 in Data:
            temp_val = self.y_n
            self.y_n = self.new_info*x_0 + self.old_info*self.y_n1
            self.y_n1 = temp_val
            temp_vector.append(self.y_n)
        return temp_vector

class exponential_signal_filter:
    def __init__(self,_a=0.5):
        self.a = _a

    def filter(self,Data:pd.DataFrame):
        temp_vector = []
        self.y_n=Data[0]
        self.y_n1=Data[0]
        for x_n in Data:
            temp_val = self.y_n
            self.y_n = self.a*self.y_n1+(1-self.a)*x_n
            self.y_n1 = temp_val
            temp_vector.append(self.y_n)
        return temp_vector

class Dexponential_signal_filter:#doble
    def __init__(self,_a=0.5,_b=0.5):
        self.a = _a
        self.b = _b

    def filter(self,Data:pd.DataFrame):
        temp_vector = []
        self.y_n=Data[0]
        self.s_n=Data[0]/2
        self.t_n=Data[0]/2
        for x_n in Data:
            s_n1 = self.s_n
            t_n1 = self.t_n
            self.s_n = self.a*x_n+(1-self.a)*(s_n1+t_n1)
            self.t_n = self.b*(self.s_n-s_n1)+(1-self.b)*t_n1
            self.y_n = self.s_n+self.t_n
            temp_vector.append(self.y_n)
        return temp_vector
    def reset(self):
        self.y_n=1
        self.s_n=1
        self.t_n=1

class Texponential_signal_filter:#triple
    def __init__(self,_a=0.5,_b=0.5,_c=0.5):
        self.a = _a
        self.b = _b
        self.c = _c

    def filter(self,Data:pd.DataFrame):
        temp_vector = []
        self.y_n=Data[0]
        self.s_n=Data[0]/3
        self.t_n=Data[0]/3
        self.p_n=Data[0]/3
        for x_n in Data:
            s_n1 = self.s_n
            t_n1 = self.t_n
            p_n1 = self.p_n
            self.s_n = self.a*(x_n-p_n1)+(1-self.a)*(s_n1+t_n1)
            self.t_n = self.b*(self.s_n-s_n1)+(1-self.b)*t_n1
            self.p_n = self.c*(x_n-self.s_n)+(1-self.c)*p_n1
            self.y_n = self.s_n+self.t_n+self.p_n
            temp_vector.append(self.y_n)
        return temp_vector

def function_retard(data,index):
    return data.shift(index)

def get_position(x_low,x_high):
    temp_vector=[]
    for index in range(len(x_low)):
        if x_low[index]>x_high[index]:
            temp_vector.append(1)
        else:
            temp_vector.append(0)
    return temp_vector

In [142]:
company_id = "BTC-USD"
fm = finance_manager()
KF = khalman_signal_filter(0.2,0.8)
EF = exponential_signal_filter(0.6)
DEF = Dexponential_signal_filter(0.6,0.1)
TEF = Texponential_signal_filter(0.1,0.6,0.1)
fm.get_data(company_id,"1d",'2020-01-01','2022-01-01')
fm.data[company_id]["Close Khalman"]= KF.filter(fm.data[company_id]["Close"])
fm.data[company_id]["Close Exponential"]= EF.filter(fm.data[company_id]["Close"])
fm.data[company_id]["Close DExponential"]= DEF.filter(fm.data[company_id]["Close"])
fm.data[company_id]["Close TExponential"]= TEF.filter(fm.data[company_id]["Close"])
fm.data[company_id]["Close Khalman Retard"]= function_retard(fm.data[company_id]["Close Khalman"],5)
fm.data[company_id]["Close DT Exponential"]=get_position(fm.data[company_id]["Close DExponential"],fm.data[company_id]["Close TExponential"])
fm.data[company_id]["Close KE position"]=get_position(fm.data[company_id]["Close Khalman"],fm.data[company_id]["Close Exponential"])
# fm.print_data(company_id,"Close","Close Khalman","Close Exponential")

fm.data[company_id]['retornos_del_instrumento'] = fm.data[company_id]['Close'].pct_change()
fm.data[company_id]['estrategia1'] = fm.data[company_id]['Close DT Exponential'].shift(1)*fm.data[company_id]['retornos_del_instrumento']
fm.data[company_id]['estrategia2'] = fm.data[company_id]['Close KE position'].shift(1)*fm.data[company_id]['retornos_del_instrumento']
fm.data[company_id]['ret_acum'] = fm.data[company_id]['retornos_del_instrumento'].cumsum().apply(np.exp)
fm.data[company_id]['ret_acum_KE'] = fm.data[company_id]['estrategia2'].cumsum().apply(np.exp)
fm.data[company_id]['ret_acum_DT'] = fm.data[company_id]['estrategia1'].cumsum().apply(np.exp)

fm.print_data(company_id,"Close","Close DExponential","Close TExponential")
# fm.print_data(company_id,"ret_acum","ret_acum_DT","ret_acum_KE")
# fm.print_data(company_id,"ret_acum","ret_acum_est")


[*********************100%***********************]  1 of 1 completed
